In [343]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import re

## Getting page content by Selenium

We need to load html content by selenium to avoid the Beautiful Soup load until the entire webpage is loaded.

In [347]:
options = Options()
options.add_argument("--headless")

In [356]:
def get_pontuacao(text):
    """
    This functions geta a string like "'\n\npunt_Gold _0000002 2\n                                            \n'"
    and extract the part referent to the pontuation
    """
    try:
        pontuacao = text.split("\n")[2].split(" ")[-1]
    except:
        pontuacao= "TO VERIFY"
    return pontuacao

In [406]:
def get_flotilha(soup_object):
    """
    This functions gets a input like <span style="display:none;">punt_Yellow _0000003</span>
    and returns the series of this data.
    """
    if "Yellow" in str(soup_object) or "Blue" in str(soup_object) or "Red"in str(soup_object) or "Pink" in str(soup_object) or "White" in str(soup_object) or "Green" in str(soup_object) or "Emerald" in str(soup_object)  or "punt_ " in str(soup_object):
        return "G"
    elif "Bronze" in str(soup_object):
        return "B"
    elif "Silver" in str(soup_object):
        return "P"
    elif "Gold" in str(soup_object):
        return "O"
    elif "Medal" in str(soup_object):
        return "Medal Race"
    else:
        return "TO VERIFY"

In [358]:
def is_data(css_class):
    return css_class=="odd" or css_class=="even"

In [415]:
COMPETICAO = "European Championship 2023"
CATEGORIA = "470"
URL = 'https://2023europeans.470.org//en/default/races/resultsajax/id/6691/idsc2r/20124/allResults/1/handicap'

with Chrome(options=options) as browser:
    browser.get(URL)
    html = browser.page_source
soup = BeautifulSoup(html, "html.parser")

data = {
    "Id Resultado": [],
    "Id Competidor": [],
    "Nome Competidor": [],
    "Id Competição": [],
    "Classe Vela": [],
    "Pontuação Regata": [],
    "Flotilha": [],
    "Posição Geral": [],
    "Punição": [],
    "Pontuação Total": [],
    "Nett": [],
    "Nome Competição": []    
}

for row in soup.find_all(class_ = is_data):

    for idx, j in enumerate(row.findChildren("td")):
        str_obj = str(j)
        if "position" in str_obj:
            position = j.text

        elif "netpoints" in str_obj:
            net = j.text

        elif "totalpoints" in str_obj:
            totalpoints = j.text

        elif idx==4: #This is 3 or 4, depending on sumula
            name = j.text   
 

    for race in row.find_all(attrs={"rel": True}):
        flotilha = get_flotilha(race)
        pontuacao = get_pontuacao(race.text)  
        data["Id Resultado"].append(None)
        data["Id Competidor"].append(None),
        data["Nome Competidor"].append(name),
        data["Id Competição"].append(None),
        data["Classe Vela"].append(CATEGORIA),
        data["Pontuação Regata"].append(pontuacao),
        data["Posição Geral"].append(position),
        data["Flotilha"].append(flotilha)
        data["Punição"].append(None),
        data["Pontuação Total"].append(totalpoints),
        data["Nett"].append(net),
        data["Nome Competição"].append(COMPETICAO) 
df= pd.DataFrame(data)
df.to_csv(f"results/{COMPETICAO+CATEGORIA}.csv")

## Agregando resultados

In [416]:
import os

In [418]:
files = [f for f in os.listdir("scraped-data")]

In [420]:
df_init = pd.read_csv("scraped-data/"+files[0])


In [423]:
for file in files[1:]:
    df_init = pd.concat([df_init, pd.read_csv("scraped-data/"+file)])

In [428]:
df_init.to_csv("scraped-data/cluster1-html.csv")